# EGES

In [ ]:
def cnt_session(data, time_cut=30, cut_type=2):
    # 商品属性  id  被交互时间   商品种类
    sku_list = data['sku_id']
    time_list = data['action_time']
    type_list = data['type']
    session = []
    tmp_session = []
    for i, item in enumerate(sku_list):
        # 两个商品之间如果被交互的时间大于1小时，划分成不同的session
        if type_list[i] == cut_type or (i < len(sku_list)-1 and \
            (time_list[i+1] - time_list[i]).seconds/60 > time_cut) or i == len(sku_list)-1:
            tmp_session.append(item)
            session.append(tmp_session)
            tmp_session = []
        else:
            tmp_session.append(item)
    return session  # 返回多个session list

In [ ]:
node_pair = dict()
# 遍历所有session list
for session in session_list_all:
    for i in range(1, len(session)):
        # 将session共现的item存到node_pair中，用于构建item-item图
        # 将共现次数所谓边的权重，即node_pair的key为边(src_node,dst_node),value为边的权重(共现次数)
        if (session[i - 1], session[i]) not in node_pair.keys():
            node_pair[(session[i - 1], session[i])] = 1
        else:
            node_pair[(session[i - 1], session[i])] += 1

in_node_list = list(map(lambda x: x[0], list(node_pair.keys())))#计算入度
out_node_list = list(map(lambda x: x[1], list(node_pair.keys())))#计算出度
weight_list = list(node_pair.values())#权重
graph_list = list([(i,o,w) for i,o,w in zip(in_node_list,out_node_list,weight_list)])
# 通过 network 构建图结构
G = nx.DiGraph().add_weighted_edges_from(graph_list)

In [1]:
node_pair = {(2,3):10}
in_node_list = list(map(lambda x: x[0], list(node_pair.keys())))
print(in_node_list)  # 输出: [2]

[2]


: 

In [ ]:
walker = RandomWalker(G, p=args.p, q=args.q)
print("Preprocess transition probs...")
walker.preprocess_transition_probs()

In [ ]:
def preprocess_transition_probs(self):
    """预处理随即游走的转移概率"""
    G = self.G
    alias_nodes = {}
    for node in G.nodes():
        # 获取每个节点与邻居节点边上的权重
        unnormalized_probs = [G[node][nbr].get('weight', 1.0)   
                                for nbr in G.neighbors(node)]
        norm_const = sum(unnormalized_probs)
         # 对每个节点的邻居权重进行归一化
        normalized_probs = [
            float(u_prob)/norm_const for u_prob in unnormalized_probs] 
        # 根据权重创建alias表
        alias_nodes[node] = create_alias_table(normalized_probs)
    alias_edges = {}
    for edge in G.edges():
        # 获取边的alias
        alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
    self.alias_nodes = alias_nodes
    self.alias_edges = alias_edges
    return

In [ ]:
session_reproduce = walker.simulate_walks(num_walks=args.num_walks, 
            walk_length=args.walk_length, workers=4,verbose=1)

In [ ]:
def _simulate_walks(self, nodes, num_walks, walk_length,):
    walks = []
    for _ in range(num_walks):
        # 打乱所有起始节点
        random.shuffle(nodes)
        for v in nodes:
            # 根据p和q选择随机游走或者带权游走
            if self.p == 1 and self.q == 1:
                walks.append(self.deepwalk_walk(
                    walk_length=walk_length, start_node=v))
            else:
                walks.append(self.node2vec_walk(
                    walk_length=walk_length, start_node=v))
    return walks


In [ ]:
sku_side_info = pd.merge(all_skus, product_data, on='sku_id', how='left').fillna(0) # 为商品加载side information
for feat in sku_side_info.columns:
    if feat != 'sku_id':
        lbe = LabelEncoder()
        # 对side information进行编码
        sku_side_info[feat] = lbe.fit_transform(sku_side_info[feat])
    else:
        sku_side_info[feat] = sku_lbe.transform(sku_side_info[feat])

In [ ]:
def get_graph_context_all_pairs(walks, window_size):
    all_pairs = []
    for k in range(len(walks)):
        for i in range(len(walks[k])):
            # 通过窗口的方式采取正样本，具体的是，让随机游走序列的起始item与窗口内的每个item组成正样本对
            for j in range(i - window_size, i + window_size + 1):
                if i == j or j < 0 or j >= len(walks[k]):
                    continue
                else:
                    all_pairs.append([walks[k][i], walks[k][j]])
    return np.array(all_pairs, dtype=np.int32)

In [ ]:
def EGES(side_information_columns, items_columns, merge_type = "weight", share_flag=True,
        l2_reg=0.0001, seed=1024):
    # side_information 所对应的特征
    feature_columns = list(set(side_information_columns))
    # 获取输入层，查字典
    feature_encode = FeatureEncoder(feature_columns,  linear_sparse_feature=None)
    # 输入的值
    feature_inputs_list = list(feature_encode.feature_input_layer_dict.values())
    # item id  获取输入层的值
    items_Map = FeatureMap(items_columns)
    items_inputs_list = list(items_Map.feature_input_layer_dict.values())

    # 正样本的id，在softmax中需要传入正样本的id
    label_columns = [DenseFeat('label_id', 1)]
    label_Map = FeatureMap(label_columns)
    label_inputs_list = list(label_Map.feature_input_layer_dict.values())

    # 通过输入的值查side_information的embedding，返回所有side_information的embedding的list
    side_embedding_list = process_feature(side_information_columns, feature_encode)
    # 拼接  N x num_feature X Dim
    side_embeddings = Concatenate(axis=1)(side_embedding_list)

    # items_inputs_list[0] 为了查找每个item 用于计算权重的 aplha 向量
    eges_inputs = [side_embeddings, items_inputs_list[0]]

    merge_emb = EGESLayer(items_columns[0].vocabulary_size, merge_type=merge_type, 
                l2_reg=l2_reg, seed=seed)(eges_inputs)  # B * emb_dim
    
    label_idx = label_Map.feature_input_layer_dict[label_columns[0].name]
    softmaxloss_inputs = [merge_emb,label_idx]
    
    item_vocabulary_size = items_columns[0].vocabulary_size

    all_items_idx = EmbeddingIndex(list(range(item_vocabulary_size)))
    all_items_embeddings = feature_encode.embedding_layers_dict[side_information_columns[0].name](all_items_idx)

    if share_flag:
        softmaxloss_inputs.append(all_items_embeddings)
    
    output = SampledSoftmaxLayer(num_items=item_vocabulary_size, share_flage=share_flag,
              emb_dim=side_information_columns[0].embedding_dim,num_sampled=10)(softmaxloss_inputs)

    model = Model(feature_inputs_list + items_inputs_list + label_inputs_list, output)
    
    model.__setattr__("feature_inputs_list", feature_inputs_list)
    model.__setattr__("label_inputs_list", label_inputs_list)
    model.__setattr__("merge_embedding", merge_emb)
    model.__setattr__("item_embedding", get_item_embedding(all_items_embeddings,                          								items_Map.feature_input_layer_dict[items_columns[0].name]))
    return model

In [ ]:
class EGESLayer(Layer):
    def __init__(self,item_nums, merge_type="weight",l2_reg=0.001,seed=1024, **kwargs):
        super(EGESLayer, self).__init__(**kwargs)
        self.item_nums = item_nums 
        self.merge_type = merge_type   #聚合方式
        self.l2_reg = l2_reg
        self.seed = seed

    def build(self, input_shape):
        if not isinstance(input_shape, list) or len(input_shape) < 2:
            raise ValueError('`EGESLayer` layer should be called \
                on a list of at least 2 inputs')
        self.feat_nums = input_shape[0][1]
        
        if self.merge_type == "weight":
            self.alpha_embeddings = self.add_weight(
                                name='alpha_attention',
                                shape=(self.item_nums, self.feat_nums),
                                dtype=tf.float32, 
                                initializer=tf.keras.initializers.RandomUniform(minval=-1, maxval=1,                                               seed=self.seed),
                                regularizer=l2(self.l2_reg))

    def call(self, inputs, **kwargs):
        if self.merge_type == "weight": 
            stack_embedding = inputs[0]  # (B * num_feate * embedding_size)
            item_input = inputs[1]       # (B * 1)  
            alpha_embedding = tf.nn.embedding_lookup(self.alpha_embeddings, item_input) #(B * 1 * num_feate)
            alpha_emb = tf.exp(alpha_embedding) 
            alpha_i_sum = tf.reduce_sum(alpha_emb, axis=-1) 
            merge_embedding = tf.squeeze(tf.matmul(alpha_emb, stack_embedding),axis=1) / alpha_i_sum
        else:
            stack_embedding = inputs[0]  # (B * num_feate * embedding_size)
            merge_embedding = tf.squeeze(tf.reduce_mean(alpha_emb, axis=1),axis=1) # (B * embedding_size)
        
        return merge_embedding

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_config(self):
        config = {"merge_type": self.merge_type, "seed": self.seed}
        base_config = super(EGESLayer, self).get_config()
        base_config.update(config)
        return base_config
